In [1]:
from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
import os
from glob import glob 
from pprint import pprint 
import json 

## 1. RAG 검색기 (Retriever)

- RAG(Retrieval Augmented Generation)는 LLM이 답변을 생성할 때 외부 지식 소스에서 관련 정보를 검색하여 참고하도록 하는 방식임. 
- 이 '검색' 단계를 수행하는 것이 바로 검색기(Retriever)임.
- 벡터 저장소는 이 검색기를 구현하는 강력한 도구임.

### 1.1 Semantic Search (의미론적 검색)

- 단어의 단순 매칭이 아닌, **문맥적 의미**의 유사성을 기반으로 문서를 검색하는 방식임.
- 벡터 저장소와 임베딩 모델을 활용하여 구현됨.
- **장점:**
  - 동의어, 유의어, 문맥적 유사성 파악 가능.
  - 키워드가 정확히 일치하지 않아도 관련 문서 검색 가능.
- **단점:**
  - 임베딩 모델의 성능에 크게 의존함.
  - 특정 키워드가 매우 중요한 경우, 오히려 키워드 검색보다 성능이 낮을 수 있음.
  - 계산 비용이 키워드 검색보다 높을 수 있음.

`(1) 벡터 저장소 초기화 (의미론적 검색용)`

- **과정:** 텍스트 파일 로드 -> 의미 단위로 분할 (Chunking) -> 임베딩 모델로 각 청크를 벡터로 변환 -> Chroma DB (벡터 저장소)에 저장.
- **주요 컴포넌트:**
  - `TextLoader`: 텍스트 파일을 로드해서 LangChain `Document` 객체로 만들어줌.
  - `CharacterTextSplitter.from_huggingface_tokenizer`: 그냥 글자 수로 자르는 게 아니라, 임베딩 모델이 사용하는 **토크나이저**를 기준으로 텍스트를 분할함. 이렇게 하면 모델이 이해하기 좋은 단위로 쪼개면서, 토큰 수 제한(LLM이나 임베딩 모델은 한 번에 처리할 수 있는 토큰 수가 정해져 있음)도 맞출 수 있음.
    - `separator=r"[.!?]\s+"`: 정규 표현식으로, 문장의 끝(마침표, 느낌표, 물음표 뒤에 공백)을 기준으로 문서를 나눔. 문맥 유지를 위해 중요!
    - `chunk_size`, `chunk_overlap`: 분할된 청크의 최대 토큰 수와 청크 간 중복될 토큰 수를 지정. `chunk_overlap`은 문맥이 갑자기 끊기는 걸 방지해 줌.
  - `Chroma` DB 생성 시 `collection_metadata = {'hnsw:space': 'cosine'}`:
    - 벡터들 사이의 유사도를 계산할 때 **코사인 유사도(cosine similarity)**를 사용하라고 명시하는 것임.
    - 코사인 유사도는 벡터의 크기보다는 방향에 중점을 둠. 그래서 텍스트 내용이 비슷하면 벡터 방향도 비슷해지는 원리를 이용해 유사도를 측정함 (0~1 사이 값, 1에 가까울수록 유사).

**꿀팁:** `chunk_size`와 `chunk_overlap`은 정답이 없음. 데이터 특성과 사용하는 모델에 맞춰 실험을 통해 최적값을 찾아야 함. 보통 `chunk_size`는 임베딩 모델의 최대 입력 토큰 수보다 작게 설정하고, `chunk_overlap`은 `chunk_size`의 10~20% 정도로 시작해 보는 것이 좋음.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoTokenizer

In [4]:
# 데이터 로드 함수
def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        print(f"로딩 중: {text_file}")
        # encoding='utf-8' 명시하여 한글 파일 로드 문제 방지
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

In [5]:
korean_txt_files = glob(os.path.join('../data', '*_KR.txt'))
if not korean_txt_files:
    print("'data' 폴더에 '*_KR.txt' 파일이 없습니다. 예시 데이터를 생성하거나 경로를 확인하세요.")
    korean_data = [] # 빈 리스트로 초기화
else:
    korean_data = load_text_files(korean_txt_files)

print(f"\n로드된 전체 문서 수: {len(korean_data)}")
if korean_data:
    print(f"첫 번째 문서 내용 일부: {korean_data[0].page_content[:100]}...")


로딩 중: ../data\Rivian_KR.txt
로딩 중: ../data\Tesla_KR.txt

로드된 전체 문서 수: 2
첫 번째 문서 내용 일부: 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" S...


In [6]:
# Hugging Face 임베딩 모델(BAAI/bge-m3)이 사용하는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 문장 구분자(정규식)를 사용하여 텍스트 분할기 생성
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,          # 토큰 수 계산 기준
    separator=r"[.!?]\s+",     # 문장 구분자: 마침표, 느낌표, 물음표 뒤 공백
    chunk_size=100,             # 청크 최대 토큰 수 (토크나이저 기준)
    chunk_overlap=20,            # 청크 간 중복 토큰 수
    is_separator_regex=True,    # separator가 정규식임을 명시
    keep_separator=True,        # 구분자 유지 여부
)

korean_docs = []
if korean_data: 
    korean_docs = text_splitter.split_documents(korean_data)

print(f"\n분할된 한국어 문서 수: {len(korean_docs)}")
if korean_docs:
    print(f"첫 번째 분할 문서 내용: {korean_docs[0].page_content}")
    print(f"첫 번째 분할 문서 메타데이터: {korean_docs[0].metadata}")


분할된 한국어 문서 수: 5
첫 번째 분할 문서 내용: 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임
첫 번째 분할 문서 메타데이터: {'source': '../data\\Rivian_KR.txt'}


In [7]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Hugging Face 임베딩 모델 재사용
embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma 벡터 저장소 생성 (코사인 유사도 사용 명시)
if korean_docs: # 분할된 문서가 있을 경우에만 DB 생성
    chroma_db_semantic = Chroma.from_documents(
        documents=korean_docs,
        embedding=embeddings_huggingface,
        collection_name="db_korean_cosine", # 새 컬렉션 이름
        persist_directory="./chroma_db_semantic", # 새 저장 경로
        collection_metadata={'hnsw:space': 'cosine'}, # 유사도 계산 방식: 코사인
    )
    print("의미론적 검색용 Chroma DB 생성 완료.")
else:
    print("분할된 문서가 없어 의미론적 검색용 Chroma DB를 생성하지 않았습니다.")
    chroma_db_semantic = None # DB가 생성되지 않았음을 명시

의미론적 검색용 Chroma DB 생성 완료.


`(2) Top K 검색`

- **방식:** `vector_store.as_retriever(search_kwargs={"k": N})` 형태로 검색기를 만듦.
- **동작:** 사용자의 질문(쿼리)과 가장 유사한 문서(청크)를 **상위 N개** 골라주는 가장 기본적인 검색 방식임.

- **장점:**
  - **단순명료:** 사용법이 아주 쉽고 직관적임.
  - **빠름:** 대부분의 벡터 DB에서 기본적으로 잘 최적화되어 있음.

- **단점:**
  - **다양성 부족 가능성:** 찾아온 문서들이 내용상으로는 다 비슷비슷할 수 있음 (예: 같은 주제의 다른 측면을 놓칠 수 있음).
  - **최적 K값 찾기:** `k`를 너무 작게 하면 중요한 정보를 놓치고, 너무 크게 하면 관련 없는 정보까지 딸려올 수 있음. 적절한 `k`값 설정이 필요함.

**꿀팁:** `k`값은 보통 3~5개로 시작해서, 결과물의 품질을 보면서 조절하는 것이 일반적임. LLM의 컨텍스트 윈도우 크기도 고려해야 함 (너무 많은 문서를 넘기면 LLM이 힘들어함).

In [8]:
if chroma_db_semantic:
    chroma_k_retriever = chroma_db_semantic.as_retriever(
        search_kwargs={"k": 2}, # 상위 2개 문서 검색
    )

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_k = chroma_k_retriever.invoke(query)

    print(f"쿼리: {query}")
    print("Top K 검색 결과:")
    for doc in retrieved_docs_k:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 Top K 검색을 수행할 수 없습니다.")

쿼리: 리비안은 언제 사업을 시작했나요?
Top K 검색 결과:
- 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임 [출처: ../data\Rivian_KR.txt]
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [출처: ../data\Rivian_KR.txt]


`(3) 임계값 지정 검색 (Similarity Score Threshold)`

- **방식:** `search_type='similarity_score_threshold'` 와 `search_kwargs={'score_threshold': T, 'k': N}`을 사용함.
- **동작:** 코사인 유사도 점수가 내가 설정한 **임계값(Threshold, T)** 이상인 문서들 중에서만, 상위 N개를 골라줌. 아무리 순위가 높아도, 유사도 점수가 기준 미달이면 탈락!
  (ChromaDB의 경우 코사인 유사도가 아닌 거리(distance)를 사용할 수 있으며, 이 경우 `score_threshold`의 의미가 반대가 됨. LangChain은 이를 내부적으로 처리하여 '유사도' 기준으로 동작하게끔 하는 경우가 많으니, 사용 중인 DB와 LangChain 버전에 따라 확인 필요. 여기서는 '높을수록 유사'한 점수 기준으로 설명)

- **장점:**
  - **관련성 필터링:** 쿼리와 전혀 상관없는 문서는 애초에 걸러낼 수 있어서 검색 결과의 질을 높일 수 있음.
  - **결과 수 조절:** 관련 있는 문서가 적으면 적게, 많으면 `k`개까지 가져와서 유동적임.

- **단점:**
  - **임계값 설정의 어려움:** 적절한 임계값(threshold) 설정이 핵심인데, 이게 좀 까다로울 수 있음. 너무 높으면 진짜 필요한 문서도 놓치고, 너무 낮으면 Top-K랑 별 차이 없거나 의미 없는 문서가 포함될 수 있음.
  - **결과 없음 가능성:** 모든 문서가 임계값을 넘지 못하면 아무것도 못 가져올 수 있음.

**꿀팁:** 어떤 임계값이 적절한지는 데이터와 쿼리의 특성에 따라 다름. 다양한 쿼리로 테스트해보면서 유사도 점수 분포를 확인하고, 적절한 임계값을 감으로 익히거나, 실험을 통해 찾아야 함. 보통 0.7~0.8 이상이면 꽤 유사하다고 봄 (사용하는 임베딩 모델에 따라 다름).

In [9]:
from langchain_community.utils.math import cosine_similarity # 코사인 유사도 계산 함수

if chroma_db_semantic:
    chroma_threshold_retriever = chroma_db_semantic.as_retriever(
        search_type='similarity_score_threshold', # 유사도 점수 임계값 기반 검색
        search_kwargs={'score_threshold': 0.5, 'k': 3}, # 코사인 유사도 0.5 이상, 상위 2~3개
    )

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_threshold = chroma_threshold_retriever.invoke(query)

    print(f"쿼리: {query}")
    print("임계값 지정 검색 결과:")
    for doc in retrieved_docs_threshold:
        # 검색 결과 문서와 쿼리 간의 코사인 유사도 직접 계산 (확인용)
        query_embedding = embeddings_huggingface.embed_query(query)
        doc_embedding = embeddings_huggingface.embed_query(doc.page_content)
        score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        print(f"- {doc.page_content} [유사도: {score:.4f}, 출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 임계값 지정 검색을 수행할 수 없습니다.")

쿼리: 리비안은 언제 사업을 시작했나요?
임계값 지정 검색 결과:
- 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임 [유사도: 0.6236, 출처: ../data\Rivian_KR.txt]
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [유사도: 0.5227, 출처: ../data\Rivian_KR.txt]


`(4) MMR (Maximal Marginal Relevance) 검색`

- `search_type='mmr'` 와 `search_kwargs={'k': N, 'fetch_k': M, 'lambda_mult': L}`을 사용.
- 검색 결과의 **정확성(유사도)**과 **다양성**을 모두 고려하는 방식임.
  - 먼저 `fetch_k` 만큼의 후보 문서를 가져온 후, MMR 알고리즘을 사용하여 최종 `k`개의 문서를 선택함.
  - `lambda_mult` (0~1 사이 값): 0에 가까울수록 다양성 중시, 1에 가까울수록 유사도(정확성) 중시. 기본값은 0.5.
- **장점:** 유사하면서도 다양한 관점의 문서를 함께 제공하여 사용자에게 더 풍부한 정보 제공 가능.
- **단점:** `fetch_k`, `lambda_mult` 등 추가 파라미터 튜닝이 필요할 수 있음. 일반 유사도 검색보다 계산량이 많을 수 있음.

In [10]:
if chroma_db_semantic:
    chroma_mmr_retriever = chroma_db_semantic.as_retriever(
        search_type='mmr', # MMR 검색 방식
        search_kwargs={
            'k': 3,             # 최종 반환할 문서 수
            'fetch_k': 8,      # MMR 계산을 위해 초기에 가져올 문서 수 (k보다 커야 함)
            'lambda_mult': 0.5, # 다양성 vs 유사도 가중치 (0: 최대 다양성, 1: 최소 다양성)
        },
    )

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_mmr = chroma_mmr_retriever.invoke(query)

    print(f"쿼리: {query}")
    print("MMR 검색 결과:")
    for doc in retrieved_docs_mmr:
        query_embedding = embeddings_huggingface.embed_query(query)
        doc_embedding = embeddings_huggingface.embed_query(doc.page_content)
        score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        print(f"- {doc.page_content} [유사도: {score:.4f}, 출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 MMR 검색을 수행할 수 없습니다.")

Number of requested results 8 is greater than number of elements in index 5, updating n_results = 5


쿼리: 리비안은 언제 사업을 시작했나요?
MMR 검색 결과:
- 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임 [유사도: 0.6236, 출처: ../data\Rivian_KR.txt]
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [유사도: 0.5227, 출처: ../data\Rivian_KR.txt]
- 테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음 [유사도: 0.3087, 출처: ../data\Tesla_KR.txt]


`(5) 메타데이터 필터링 검색`

- `search_kwargs={'filter': {'metadata_key': 'metadata_value'}, 'k': N}` 형태로 사용.
- 문서에 저장된 메타데이터를 기준으로 검색 대상을 한정한 후, 그 안에서 유사도 높은 N개 문서를 찾음.
- **장점:** 특정 출처, 카테고리 등 원하는 조건의 문서 내에서만 검색 가능하여 효율적이고 정확한 검색 가능.
- **단점:** 메타데이터가 잘 정의되고 일관성 있게 관리되어야 효과적임.

In [ ]:
if chroma_db_semantic:
    target_source_file = '../data/Rivian_KR.txt' 
    if not os.path.exists(target_source_file):
        print(f"경고: '{target_source_file}' 파일이 없어 메타데이터 필터링이 제대로 동작하지 않을 수 있습니다.")

    chroma_metadata_retriever = chroma_db_semantic.as_retriever(
        search_kwargs={
            'filter': {'source': target_source_file}, # 'source' 메타데이터가 특정 파일인 문서만 대상
            'k': 1, 
        }
    )

    query = "리비안의 주요 모델은 무엇인가요?"
    retrieved_docs_metadata = chroma_metadata_retriever.invoke(query)

    print(f"쿼리: {query}")
    print(f"메타데이터 필터링 검색 결과 (출처: {target_source_file}):")
    for doc in retrieved_docs_metadata:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("Chroma DB (semantic)가 초기화되지 않아 메타데이터 필터링 검색을 수행할 수 없습니다.")

경고: './data/Rivian_KR.txt' 파일이 없어 메타데이터 필터링이 제대로 동작하지 않을 수 있습니다.
쿼리: 리비안의 주요 모델은 무엇인가요?
메타데이터 필터링 검색 결과 (출처: ./data/Rivian_KR.txt):


### 1.2 Keyword Search (키워드 검색)

- **정의:** TF-IDF, BM25 같은 전통적인 정보 검색 알고리즘을 사용함. 사용자가 입력한 쿼리에 포함된 **키워드**와 문서 내 **키워드**가 얼마나 일치하는지를 기준으로 문서를 찾아줌.

- **장점:**
  - **속도 빠름 & 저비용:** 계산이 단순해서 일반적으로 의미론적 검색보다 빠르고 컴퓨팅 자원도 적게 듬.
  - **키워드 정확도:** 특정 키워드가 반드시 포함되어야 하는 검색(예: 제품명, 법률 용어)에서는 의미론적 검색보다 오히려 더 정확할 수 있음.
  - **구현 용이:** 상대적으로 구현이 간단함.

- **단점:**
  - **문맥 이해 불가:** 동의어, 유의어 같은 의미론적 유사성은 전혀 파악 못 함 (예: '자동차'로 검색하면 '탈것'이란 단어가 포함된 문서는 못 찾음).
  - **단어 순서 무시:** 단어의 순서나 문장 구조를 고려하지 않음.
  - **한국어 처리 어려움:** 한국어는 조사가 붙고 어미가 변하는 등 형태가 다양해서, 단순 공백 기준으로 단어를 나누면(토큰화하면) 성능이 바닥을 침. 그래서 좋은 **한국어 형태소 분석기(토크나이저)**가 필수임!

**꿀팁:**
  - 키워드 검색은 의미론적 검색과 **상호 보완적**으로 사용될 때 시너지가 남 (하이브리드 검색).
  - 한국어의 경우, 어떤 형태소 분석기를 쓰느냐에 따라 검색 품질이 하늘과 땅 차이일 수 있음. `Kiwi`, `Mecab`, `Okt` 등이 있음.

`(1) BM25 검색기 생성`

- **BM25 (Best Matching 25)란?** TF-IDF를 개선한 알고리즘으로, 문서 길이까지 고려해서 점수를 매기는 방식. 키워드 검색계의 표준 같은 존재임.
- `BM25Retriever.from_documents(docs)`: LangChain의 `Document` 객체 리스트로부터 BM25 검색기를 쉽게 만들 수 있음.
- **기본 토큰화 방식:** 특별히 지정 안 하면, 그냥 **공백**을 기준으로 단어를 분리(토큰화)해서 사용함. 영어에는 그럭저럭 괜찮지만, 한국어에는 매우 부적합!

- **장점 (기본 BM25):**
  - **간단함:** 설정할 게 거의 없이 바로 사용 가능.
  - **빠름:** 여전히 빠름.

- **단점 (기본 BM25, 특히 한국어에서):**
  - **한국어 성능 저하:** '리비안은'과 '리비안이'를 다른 단어로 인식함. 조사, 어미 변화에 매우 취약.
  - **의미 파악 불가:** 여전히 단어 뜻은 모름.

**꿀팁:** 아래 예제에서 보겠지만, 한국어 텍스트에 기본 BM25를 쓰면 결과가 처참할 수 있음. '아, 이래서 한국어 토크나이저가 중요하구나!'를 느끼는 계기가 될 것임.

In [12]:
if chroma_db_semantic:
    bm25_input_docs = korean_docs # 이미 Document 객체 리스트임
    print(f"BM25 검색기 생성을 위한 문서 수: {len(bm25_input_docs)}")

    from langchain_community.retrievers import BM25Retriever
    
    bm25_retriever_simple = BM25Retriever.from_documents(bm25_input_docs)
    print("기본 BM25 검색기 생성 완료.")

    query = "리비안은 언제 사업을 시작했나요?"
    retrieved_docs_bm25_simple = bm25_retriever_simple.invoke(query)

    print(f"\n쿼리: {query}")
    print("기본 BM25 검색 결과:")
    for doc in retrieved_docs_bm25_simple:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("korean_docs가 없어 BM25 검색기를 생성할 수 없습니다.")
    bm25_retriever_simple = None

BM25 검색기 생성을 위한 문서 수: 5
기본 BM25 검색기 생성 완료.

쿼리: 리비안은 언제 사업을 시작했나요?
기본 BM25 검색 결과:
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [출처: ../data\Rivian_KR.txt]
- .
또한, 테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Powerwall), 대규모 에너지 저장 시스템인 메가팩(Megapack) 등 에너지 관련 제품도 생산하고 있음.
전기차 시장의 혁신을 이끌고 있으며, 자율 주행 기술 개발에도 많은 투자를 하고 있음. [출처: ../data\Tesla_KR.txt]
- . 본사는 텍사스주 오스틴에 있음.
테슬라의 대표적인 전기차 모델로는 모델 S (세단), 모델 3 (세단), 모델 X (SUV), 모델 Y (컴팩트 SUV), 그리고 사이버트럭 (픽업트럭) 등이 있음 [출처: ../data\Tesla_KR.txt]
- 테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음 [출처: ../data\Tesla_KR.txt]


In [13]:
if bm25_retriever_simple:
    # BM25 점수 확인 (내부 vectorizer 사용)
    query_bm25_score_check = "리비안은 언제 사업을 시작했나요?"
    tokenized_query_bm25 = query_bm25_score_check.split() # 기본 공백 토큰화
    print(f"토큰화된 쿼리 (공백 기준): {tokenized_query_bm25}")

    print("BM25 점수는 invoke 결과의 순서로 유추할 수 있음 (높은 순으로 정렬됨).")
else:
    print("BM25 검색기가 없어 점수를 확인할 수 없습니다.")

토큰화된 쿼리 (공백 기준): ['리비안은', '언제', '사업을', '시작했나요?']
BM25 점수는 invoke 결과의 순서로 유추할 수 있음 (높은 순으로 정렬됨).


In [14]:
# 의미는 같지만 다른 단어를 사용한 쿼리 테스트
if bm25_retriever_simple:
    query_synonym = "리비안이 설립된 연도는?" # '사업 시작' -> '설립 연도'
    retrieved_docs_bm25_synonym = bm25_retriever_simple.invoke(query_synonym)

    print(f"쿼리: {query_synonym}")
    print("유의어 사용 시 기본 BM25 검색 결과:")
    for doc in retrieved_docs_bm25_synonym:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("\n-> 기본 BM25는 '설립'과 '시작'의 의미적 유사성을 모르므로, 키워드가 다르면 결과가 달라짐.")
else:
    print("BM25 검색기가 없어 유의어 테스트를 수행할 수 없습니다.")

쿼리: 리비안이 설립된 연도는?
유의어 사용 시 기본 BM25 검색 결과:
- .
또한, 테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Powerwall), 대규모 에너지 저장 시스템인 메가팩(Megapack) 등 에너지 관련 제품도 생산하고 있음.
전기차 시장의 혁신을 이끌고 있으며, 자율 주행 기술 개발에도 많은 투자를 하고 있음. [출처: ../data\Tesla_KR.txt]
- . 본사는 텍사스주 오스틴에 있음.
테슬라의 대표적인 전기차 모델로는 모델 S (세단), 모델 3 (세단), 모델 X (SUV), 모델 Y (컴팩트 SUV), 그리고 사이버트럭 (픽업트럭) 등이 있음 [출처: ../data\Tesla_KR.txt]
- 테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.
2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음 [출처: ../data\Tesla_KR.txt]
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [출처: ../data\Rivian_KR.txt]

-> 기본 BM25는 '설립'과 '시작'의 의미적 유사성을 모르므로, 키워드가 다르면 결과가 달라짐.


`(2) Kiwi 한국어 토크나이저 활용 BM25` - 한국어 패치 완료!

- **왜 필요한가?** 한국어는 조사(은/는/이/가, 을/를), 어미 변화(-했다, -했음, -하는) 등이 매우 발달해서, 단순 공백 기준 토큰화로는 '리비안'과 '리비안은'을 다른 단어로 취급함. 이러면 검색 성능이 나락으로 감.
- **해결책:** `kiwipiepy` 같은 한국어 형태소 분석기를 사용해서, 단어를 의미 있는 최소 단위(형태소)로 잘 쪼개고, 기본형으로 바꾸거나 불필요한 조사를 제거하는 등의 전처리를 해야 함.
- **적용법:** `BM25Retriever`의 `preprocess_func` 인자에 우리가 직접 만든 토큰화 함수를 전달하면 됨.
- **사용자 사전(`add_user_word`):** '리비안', '테슬라', '챗GPT' 같은 신조어나 고유명사를 형태소 분석기가 제대로 인식하도록 사전에 추가해 줄 수 있음. 이러면 '리비안은'을 '리비안' + '은'(조사)으로 정확히 분리 가능.

- **장점 (Kiwi + BM25):**
  - **한국어 검색 정확도 대폭 향상:** 조사, 어미 변화에 강해져서 동일한 의미의 다른 표현도 잘 찾아낼 가능성이 높아짐.
  - **사용자 사전 활용:** 특정 도메인의 고유명사나 신조어도 잘 처리하도록 커스터마이징 가능.

- **단점 (Kiwi + BM25):**
  - **형태소 분석 오버헤드:** 형태소 분석기를 로드하고 실행하는 데 약간의 시간과 자원이 추가로 듦 (문서가 아주 많을 경우).
  - **토크나이저 의존성:** 여전히 Kiwi 형태소 분석기의 성능에 의존적임. 완벽하진 않음.
  - **여전히 비의미론적:** 단어의 의미 자체를 이해하는 건 아님. 키워드 기반이라는 한계는 여전.

**꿀팁:**
  - `Kiwi()` 객체는 `preprocess_func` 안에서 매번 생성하면 느림. 실제 서비스에서는 한 번만 생성해서 재사용하거나, 클래스 멤버로 관리하는 것이 좋음 (예제에서는 간결성을 위해 함수 내 생성).
  - 사용자 사전은 꾸준히 관리해 줘야 성능 유지에 도움이 됨.

In [15]:
# !pip install kiwipiepy # 설치 필요 시 주석 해제
from kiwipiepy import Kiwi # Kiwi 형태소 분석기

# Kiwi 토크나이저를 사용한 전처리 함수
def kiwi_bm25_process_func(text):
    kiwi = Kiwi() # 함수 호출 시마다 Kiwi 객체 생성 (성능 고려 시 외부에서 생성 후 전달 권장)
    # 사용자 사전 추가 (고유명사 등)
    kiwi.add_user_word('리비안', 'NNP') # NNP: 고유명사 품사 태그
    kiwi.add_user_word('테슬라', 'NNP')
    # 텍스트를 토큰화하여 형태소(form) 리스트 반환
    return [token.form for token in kiwi.tokenize(text)]

if korean_docs: # 이전 단계의 korean_docs 사용
    bm25_retriever_kiwi = BM25Retriever.from_documents(
        documents=korean_docs,
        preprocess_func=kiwi_bm25_process_func, # Kiwi 토큰화 함수 사용
    )
    print("Kiwi 토크나이저 적용 BM25 검색기 생성 완료.")

    # 이전과 동일한 유의어 쿼리로 검색
    query_synonym_kiwi = "리비안이 설립된 연도는?"
    retrieved_docs_bm25_kiwi = bm25_retriever_kiwi.invoke(query_synonym_kiwi)

    print(f"\n쿼리: {query_synonym_kiwi}")
    print("Kiwi 토크나이저 적용 BM25 검색 결과:")
    for doc in retrieved_docs_bm25_kiwi:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("\n-> Kiwi를 사용하면 '설립', '연도' 등의 키워드가 더 잘 분리되어 매칭될 가능성 높아짐.")
    
    # Kiwi 토크나이저로 쿼리가 어떻게 토큰화되는지 확인
    kiwi_instance = Kiwi()
    kiwi_instance.add_user_word('리비안', 'NNP')
    tokenized_query_kiwi_check = [t.form for t in kiwi_instance.tokenize(query_synonym_kiwi)]
    print(f"Kiwi로 토큰화된 쿼리: {tokenized_query_kiwi_check}")
else:
    print("korean_docs가 없어 Kiwi 적용 BM25 검색기를 생성할 수 없습니다.")
    bm25_retriever_kiwi = None

Kiwi 토크나이저 적용 BM25 검색기 생성 완료.

쿼리: 리비안이 설립된 연도는?
Kiwi 토크나이저 적용 BM25 검색 결과:
- 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임 [출처: ../data\Rivian_KR.txt]
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [출처: ../data\Rivian_KR.txt]
- . 본사는 텍사스주 오스틴에 있음.
테슬라의 대표적인 전기차 모델로는 모델 S (세단), 모델 3 (세단), 모델 X (SUV), 모델 Y (컴팩트 SUV), 그리고 사이버트럭 (픽업트럭) 등이 있음 [출처: ../data\Tesla_KR.txt]
- .
또한, 테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Powerwall), 대규모 에너지 저장 시스템인 메가팩(Megapack) 등 에너지 관련 제품도 생산하고 있음.
전기차 시장의 혁신을 이끌고 있으며, 자율 주행 기술 개발에도 많은 투자를 하고 있음. [출처: ../data\Tesla_KR.txt]

-> Kiwi를 사용하면 '설립', '연도' 등의 키워드가 더 잘 분리되어 매칭될 가능성 높아짐.
Kiwi로 토큰화된 쿼리: ['리비안', '이', '설립', '되', 'ᆫ', '연도', '는', '?']


### 3.3 Hybrid Search (하이브리드 검색)

- 의미론적 검색(Semantic Search)과 키워드 검색(Keyword Search)의 장점을 결합하는 방식임.
- `EnsembleRetriever`를 사용하여 여러 검색기(예: Chroma 검색기, BM25 검색기)를 하나로 묶고, 각 검색 결과에 가중치(`weights`)를 부여하여 최종 순위를 결정함.

- **장점:**
  - **성능 끝판왕 (잠재력):** 의미론적 유사성(문맥 파악)과 키워드 정확성(특정 단어 일치)을 모두 잡을 수 있어서, 잘만 튜닝하면 각개 검색 방식보다 훨씬 뛰어난 성능을 낼 수 있음.
  - **상호 보완:** 한쪽 검색 방식이 놓치는 부분을 다른 쪽이 커버해 줌. (예: 의미론적 검색이 특정 제품 코드를 놓치면, 키워드 검색이 잡아줌).

- **단점:**
  - **계산 비용 증가:** 여러 검색기를 돌리니까 당연히 시간과 자원이 더 듬.
  - **튜닝의 복잡성:** 각 검색기 자체도 튜닝해야 하는데, 앙상블 가중치(`weights`)나 퓨전 방식까지 신경 써야 하니 최적값 찾기가 더 어려울 수 있음.
  - **구현 및 관리 복잡도 증가:** 시스템이 더 복잡해짐.


**꿀팁:**
  - 가중치(`weights`)는 보통 모든 검색기에 동일하게 [0.5, 0.5] 또는 [0.33, 0.33, 0.33] 같이 시작해서, 결과물을 보면서 조절함. 어떤 검색기가 더 중요하다고 판단되면 그쪽에 가중치를 더 줄 수 있음.
  - 하이브리드 검색은 특히 사용자의 다양한 검색 의도(넓은 의미의 검색 vs 특정 키워드 검색)에 모두 잘 대응해야 할 때 유용함.
  - `EnsembleRetriever`는 기본적으로 가져온 문서들의 점수를 합산하는 방식 외에 `c` 파라미터를 통해 Reciprocal Rank Fusion (RRF) 점수를 사용하도록 할 수도 있음. RRF는 다른 스케일의 점수를 가진 검색기들을 결합할 때 유용함.

In [16]:
from langchain.retrievers import EnsembleRetriever

# 이전 단계에서 생성한 검색기들을 사용
# chroma_threshold_retriever (의미론적 검색기, 임계값 기반)
# bm25_retriever_kiwi (키워드 검색기, Kiwi 토크나이저 사용)

if chroma_threshold_retriever and bm25_retriever_kiwi: # 두 검색기가 모두 준비되었을 때만 실행
    ensemble_retriever = EnsembleRetriever(
        retrievers=[chroma_threshold_retriever, bm25_retriever_kiwi],
        weights=[0.5, 0.5]  # 각 검색기 결과에 대한 가중치 (합이 1이 아니어도 됨, 상대적 비율)
    )
    print("Ensemble Retriever (하이브리드 검색기) 생성 완료.")

    query_hybrid = "리비안이 설립된 연도는?"
    retrieved_docs_hybrid = ensemble_retriever.invoke(query_hybrid)

    print(f"\n쿼리: {query_hybrid}")
    print("하이브리드 검색 결과:")
    # EnsembleRetriever는 중복된 문서를 자동으로 제거해줌
    for doc in retrieved_docs_hybrid:
        print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
else:
    print("의미론적 검색기 또는 BM25 검색기가 준비되지 않아 하이브리드 검색기를 생성할 수 없습니다.")

Ensemble Retriever (하이브리드 검색기) 생성 완료.

쿼리: 리비안이 설립된 연도는?
하이브리드 검색 결과:
- 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임 [출처: ../data\Rivian_KR.txt]
- . 이들 차량은 "스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조함.
리비안은 아마존(Amazon)과 포드(Ford) 등 주요 기업으로부터 투자를 유치했으며, 아마존에는 전기 배송 밴을 공급하는 계약을 체결하기도 했음.
2021년 말에 나스닥에 상장되어 큰 주목을 받았음. [출처: ../data\Rivian_KR.txt]
- . 본사는 텍사스주 오스틴에 있음.
테슬라의 대표적인 전기차 모델로는 모델 S (세단), 모델 3 (세단), 모델 X (SUV), 모델 Y (컴팩트 SUV), 그리고 사이버트럭 (픽업트럭) 등이 있음 [출처: ../data\Tesla_KR.txt]
- .
또한, 테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Powerwall), 대규모 에너지 저장 시스템인 메가팩(Megapack) 등 에너지 관련 제품도 생산하고 있음.
전기차 시장의 혁신을 이끌고 있으며, 자율 주행 기술 개발에도 많은 투자를 하고 있음. [출처: ../data\Tesla_KR.txt]


**파라미터 튜닝은 숙명:**
    - `k` (가져올 문서 수), `score_threshold` (유사도/거리 임계값), `fetch_k` 및 `lambda_mult` (MMR용), `weights` (앙상블용) 등등... 수많은 파라미터들이 있음.
    - 정답은 없음! 데이터와 사용자의 질의 패턴에 맞춰 **끊임없이 실험하고 평가**하면서 최적값을 찾아나가야 함.